# Qwen3-VL Training Pipeline on Google Colab

This notebook provides training for Qwen3-VL with:
- **SFT** (Supervised Fine-Tuning)
- **DPO** (Direct Preference Optimization)
- **GRPO** (Group Relative Policy Optimization)

**Requirements:**
- GPU runtime (T4 free tier, or V100/A100 Pro)
- Google Drive for checkpoints (recommended)
- Weights & Biases account (optional)

## Quick Start
1. Select GPU runtime: `Runtime` > `Change runtime type` > `T4 GPU`
2. Run cells in order
3. Monitor training in WandB (optional)

## 1. GPU Check

In [ ]:
# Check GPU availability and type
!nvidia-smi

import torch

print(f"\nCUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"VRAM: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")
else:
    print("WARNING: No GPU detected. Training will be very slow.")
    print("Go to Runtime > Change runtime type > T4 GPU")

## 2. Mount Google Drive

Mounting Drive ensures checkpoints and datasets persist across sessions.

In [ ]:
from google.colab import drive
import os

# Mount Google Drive
drive.mount("/content/drive")

# Define paths
DRIVE_BASE = "/content/drive/MyDrive/qwen3_vl_training"
CHECKPOINTS_DIR = f"{DRIVE_BASE}/checkpoints"
DATASETS_CACHE = f"{DRIVE_BASE}/datasets_cache"
HF_CACHE = f"{DRIVE_BASE}/hf_cache"

# Create directories
for dir_path in [CHECKPOINTS_DIR, DATASETS_CACHE, HF_CACHE]:
    os.makedirs(dir_path, exist_ok=True)

# Set HuggingFace cache to Drive for persistence
os.environ["HF_HOME"] = HF_CACHE
os.environ["HF_DATASETS_CACHE"] = DATASETS_CACHE
os.environ["TRANSFORMERS_CACHE"] = HF_CACHE

print(f"Checkpoints: {CHECKPOINTS_DIR}")
print(f"Dataset cache: {DATASETS_CACHE}")
print(f"HF cache: {HF_CACHE}")

## 3. Clone Repository and Install Dependencies

In [ ]:
import os

# Clone repository (modify URL to your fork if needed)
REPO_URL = "https://github.com/teenylasers/qwenvl_sandbox.git"
REPO_DIR = "/content/qwenvl_sandbox"

if not os.path.exists(REPO_DIR):
    !git clone {REPO_URL} {REPO_DIR}
else:
    print(f"Repository already exists at {REPO_DIR}")
    !cd {REPO_DIR} && git pull

%cd {REPO_DIR}

# Install dependencies
!pip install -e ".[cloud]" -q

print("\nInstallation complete!")

## 4. WandB Authentication (Optional)

Weights & Biases provides experiment tracking. You can:
1. Add `WANDB_API_KEY` to Colab Secrets (Settings > Secrets)
2. Login interactively below
3. Skip and set `USE_WANDB = False`

In [ ]:
import wandb

USE_WANDB = True  # Set to False to disable

if USE_WANDB:
    try:
        # Method 1: Try Colab Secrets
        from google.colab import userdata

        wandb_key = userdata.get("WANDB_API_KEY")
        wandb.login(key=wandb_key)
        print("Logged in via Colab Secrets")
    except Exception as e:
        # Method 2: Interactive login
        print("Colab Secret not found. Trying interactive login...")
        try:
            wandb.login()
            print("Logged in interactively")
        except Exception as e2:
            print(f"WandB login failed: {e2}")
            print("Set USE_WANDB = False to continue without logging")
            USE_WANDB = False
else:
    print("WandB disabled")

## 5. Training Configuration

Adjust these settings based on your GPU and requirements.

In [ ]:
# @title Training Configuration {display-mode: "form"}

# @markdown ### Training Stage
TRAINING_STAGE = "sft"  # @param ["sft", "dpo", "grpo"]

# @markdown ### Dataset Options
MAX_SAMPLES = 1000  # @param {type:"integer"}
DATASETS = ["rlhfv"]  # @param {type:"raw"}

# @markdown ### Training Options
NUM_EPOCHS = 1  # @param {type:"integer"}
BATCH_SIZE = 1  # @param {type:"integer"}
GRADIENT_ACCUMULATION = 8  # @param {type:"integer"}
LEARNING_RATE = 2e-5  # @param {type:"number"}

# @markdown ### Checkpointing
SAVE_STEPS = 200  # @param {type:"integer"}
RESUME_FROM_CHECKPOINT = ""  # @param {type:"string"}

# @markdown ### Previous Stage Checkpoint (for DPO/GRPO)
SFT_CHECKPOINT = ""  # @param {type:"string"}

print(f"Stage: {TRAINING_STAGE}")
print(f"Samples: {MAX_SAMPLES}")
print(f"Epochs: {NUM_EPOCHS}")

## 6. Generate Configuration

Creates an optimized config based on detected GPU.

In [ ]:
import yaml
import torch


def detect_gpu_tier():
    """Detect GPU tier from hardware."""
    if not torch.cuda.is_available():
        return "cpu"
    gpu_name = torch.cuda.get_device_name(0).lower()
    if "a100" in gpu_name:
        return "a100"
    elif "v100" in gpu_name:
        return "v100"
    else:
        return "t4"


GPU_TIER = detect_gpu_tier()
print(f"Detected GPU tier: {GPU_TIER}")

# GPU-specific optimizations for 2B model (default)
# max_pixels controls vision token count per image (1 token per 28x28 patch).
# Must be sized so vision tokens fit within max_seq_length with room for text.
GPU_CONFIGS = {
    "t4": {"max_seq_length": 2048, "lora_r": 16, "grpo_generations": 4,
            "min_pixels": 128 * 28 * 28, "max_pixels": 512 * 28 * 28},
    "v100": {"max_seq_length": 1536, "lora_r": 32, "grpo_generations": 4,
             "min_pixels": 128 * 28 * 28, "max_pixels": 768 * 28 * 28},
    "a100": {"max_seq_length": 2048, "lora_r": 32, "grpo_generations": 4,
             "min_pixels": 256 * 28 * 28, "max_pixels": 1280 * 28 * 28},
    "cpu": {"max_seq_length": 512, "lora_r": 8, "grpo_generations": 1,
            "min_pixels": 128 * 28 * 28, "max_pixels": 256 * 28 * 28},
}

gpu_cfg = GPU_CONFIGS[GPU_TIER]
output_dir = f"{CHECKPOINTS_DIR}/{TRAINING_STAGE}"

# Build config
config = {
    "model": {
        "name": "Qwen/Qwen3-VL-2B-Instruct",
        "use_lora": True,
        "use_4bit": True,
        "lora": {
            "r": gpu_cfg["lora_r"],
            "alpha": 16,
            "dropout": 0.05,
        },
    },
    "data": {
        "datasets": DATASETS,
        "max_samples_per_dataset": MAX_SAMPLES,
        "image": {
            "min_pixels": gpu_cfg["min_pixels"],
            "max_pixels": gpu_cfg["max_pixels"],
        },
        "shuffle": True,
        "seed": 42,
    },
    "training": {
        "output_dir": output_dir,
        "num_epochs": NUM_EPOCHS,
        "batch_size": BATCH_SIZE,
        "gradient_accumulation_steps": GRADIENT_ACCUMULATION,
        "learning_rate": LEARNING_RATE,
        "warmup_ratio": 0.1,
        "weight_decay": 0.01,
        "max_seq_length": gpu_cfg["max_seq_length"],
        "gradient_checkpointing": True,
    },
    "hardware": {
        "bf16": GPU_TIER != "cpu",
        "dataloader_num_workers": 2,
    },
    "logging": {
        "steps": 10,
        "save_steps": SAVE_STEPS,
        "save_total_limit": 2,
        "report_to": "wandb" if USE_WANDB else "none",
        "run_name": f"qwen-vl-{TRAINING_STAGE}-colab",
    },
    "debug": {
        "enabled": False,
    },
}

# Stage-specific settings
if TRAINING_STAGE == "dpo":
    config["dpo"] = {"beta": 0.1, "loss_type": "sigmoid"}
    config["training"]["learning_rate"] = 5e-7
    if SFT_CHECKPOINT:
        config["model"]["sft_checkpoint"] = SFT_CHECKPOINT
elif TRAINING_STAGE == "grpo":
    config["grpo"] = {
        "num_generations": gpu_cfg["grpo_generations"],
        "temperature": 0.7,
        "top_p": 0.9,
        "reward_type": "combined",
    }
    config["hardware"]["use_vllm"] = False
    config["training"]["learning_rate"] = 1e-6
    if SFT_CHECKPOINT:
        config["model"]["sft_checkpoint"] = SFT_CHECKPOINT

# Save config
config_path = f"/content/qwenvl_sandbox/configs/colab_{TRAINING_STAGE}_runtime.yaml"
with open(config_path, "w") as f:
    yaml.dump(config, f, default_flow_style=False)

print(f"\nConfig saved to: {config_path}")
print(f"Output directory: {output_dir}")
print(f"Image pixels: min={gpu_cfg['min_pixels']} (~{gpu_cfg['min_pixels'] // (28*28)} tokens), "
      f"max={gpu_cfg['max_pixels']} (~{gpu_cfg['max_pixels'] // (28*28)} tokens)")
print(f"Max sequence length: {gpu_cfg['max_seq_length']}")
print("\nConfiguration:")
print(yaml.dump(config, default_flow_style=False))

## 7. Download Datasets

Datasets are cached to Google Drive for persistence.

In [ ]:
datasets_str = " ".join(DATASETS)
!python scripts/download_datasets.py --datasets {datasets_str}

## 8. Run Training

In [ ]:
import subprocess
import sys

# Build command
cmd = [
    sys.executable,
    f"scripts/train_{TRAINING_STAGE}.py",
    "--config",
    config_path,
]

if not USE_WANDB:
    cmd.append("--no_wandb")

if RESUME_FROM_CHECKPOINT:
    cmd.extend(["--resume_from_checkpoint", RESUME_FROM_CHECKPOINT])

print(f"Running: {' '.join(cmd)}")
print("=" * 50)

# Run training
process = subprocess.Popen(
    cmd, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True, bufsize=1
)

for line in iter(process.stdout.readline, ""):
    print(line, end="")

process.wait()
print("=" * 50)
print(f"Training completed with return code: {process.returncode}")

## 9. Checkpoint Management

In [ ]:
import os
import glob


def list_checkpoints(base_dir):
    """List all checkpoints in the directory."""
    pattern = os.path.join(base_dir, "**/checkpoint-*")
    checkpoints = glob.glob(pattern, recursive=True)
    return sorted(checkpoints)


print("Available checkpoints:")
print("=" * 50)
for cp in list_checkpoints(CHECKPOINTS_DIR):
    size = sum(
        os.path.getsize(os.path.join(cp, f))
        for f in os.listdir(cp)
        if os.path.isfile(os.path.join(cp, f))
    )
    print(f"  {cp}")
    print(f"    Size: {size / 1e6:.1f} MB")

print("\nTo resume, copy checkpoint path to RESUME_FROM_CHECKPOINT above")

## 10. Session Resume Helper

Run this cell after a session timeout to restore your environment.

In [ ]:
# @title Session Resume Helper {display-mode: "form"}
# @markdown Run this after a session timeout

# Re-mount Drive
from google.colab import drive

drive.mount("/content/drive", force_remount=True)

# Restore environment variables
import os

DRIVE_BASE = "/content/drive/MyDrive/qwen3_vl_training"
os.environ["HF_HOME"] = f"{DRIVE_BASE}/hf_cache"
os.environ["HF_DATASETS_CACHE"] = f"{DRIVE_BASE}/datasets_cache"
os.environ["TRANSFORMERS_CACHE"] = f"{DRIVE_BASE}/hf_cache"

# Check if repo exists, re-clone if needed
REPO_DIR = "/content/qwenvl_sandbox"
if not os.path.exists(REPO_DIR):
    print("Repository not found. Please run Cell 3 to clone it.")
else:
    %cd {REPO_DIR}
    !pip install -e ".[cloud]" -q
    print("Environment restored!")

# List available checkpoints
import glob

print("\nAvailable checkpoints to resume from:")
for cp in sorted(glob.glob(f"{DRIVE_BASE}/checkpoints/*/checkpoint-*")):
    print(f"  {cp}")

---

## Next Steps

After SFT training:
1. Set `TRAINING_STAGE = "dpo"`
2. Set `SFT_CHECKPOINT` to your SFT checkpoint path
3. Re-run cells 5-8

After DPO training:
1. Set `TRAINING_STAGE = "grpo"`
2. Set `SFT_CHECKPOINT` to your DPO checkpoint path
3. Re-run cells 5-8